# Inverse scattering for circular billiard

## Importação das bibliotecas

In [ ]:
import numpy as np
import scipy
import scipy.special as sc
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

## Lendo os dados

In [ ]:
df = pd.read_csv('./data/dados.csv')

In [ ]:
df

## Plot da scattering cross length para gamma, R = 2.0

In [ ]:
df_R_2 = df[df['R'] == 2.0]

In [ ]:
row_gamma_R_2 = df_R_2[(df_R_2['gamma'] > 1.97) & (df_R_2['gamma'] < 2.03)]

In [ ]:
row_gamma_R_2

In [ ]:
l_array = row_gamma_R_2.drop(columns=['M', 'HBAR', 'k_min', 'k_max', 'delta_k', 'n_min', 'n_max', 'gamma', 'R']).to_numpy()

In [ ]:
l_array.shape

In [ ]:
k_min = 0.02
k_max = 3.0
k = np.linspace(k_min, k_max, 596)

fig = plt.figure(figsize=(8,6))
axes = fig.add_axes([0.1,0.1,0.8,0.8])
axes.plot(k, l_array[0], ls='-', label=f"$\gamma = 1.97$")
axes.plot(k, l_array[1], ls='-', label=f"$\gamma = 1.98$")
axes.plot(k, l_array[2], ls='-', label=f"$\gamma = 1.99$")
axes.plot(k, l_array[3], ls='-', label=f"$\gamma = 2.00$")
axes.plot(k, l_array[4], ls='-', label=f"$\gamma = 2.01$")
axes.plot(k, l_array[5], ls='-', label=f"$\gamma = 2.02$")
axes.set_title(f'Scattering cross length values')
axes.set_xlabel(f'k')
axes.legend(loc='upper right')
plt.grid(linestyle='-', linewidth=0.5)

## Implementação da rede neural

### Definição dos inputs da rede

In [ ]:
features = df.drop(columns=['gamma', 'R'])

features = features.to_numpy()

In [ ]:
targets = df[['gamma', 'R']]

targets = targets.to_numpy()

In [ ]:
print(features.shape)
print(targets.shape)

### Definição da rede neural

A rede neural implementada é uma rede neural do tipo Multilayer Perceptron

In [ ]:
def calculate_hidden_neurons(input_size, output_size):
    # Rule 1: The number of hidden neurons should be between the size of the input layer and the size of the output layer
    rule_1 = max(input_size, output_size)
    
    # Rule 2: The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer
    rule_2 = int(2/3 * input_size + output_size)
    
    # Rule 3: The number of hidden neurons should be less than twice the size of the input layer
    rule_3 = min(2 * input_size - 1, input_size + input_size // 3)
    
    # The number of hidden neurons should be the minimum that satisfies all rules, so will be the input size + 1/3*input_size
    hidden_neurons = int(input_size + input_size/3)

    print(hidden_neurons)
    return 201

class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        hidden_neurons = calculate_hidden_neurons(input_size, output_size)
        
        self.hidden_layer = nn.Linear(input_size, hidden_neurons)
        self.output_layer = nn.Linear(hidden_neurons, output_size)
        self.sigmoid = nn.Sigmoid()

    # def add_hidden_layer(self):
    #     # Determine the size of the new hidden layer
    #     if len(self.hidden_layers) == 0:
    #         hidden_size = calculate_hidden_neurons(self.input_size, self.output_size)
    #         new_layer = nn.Linear(self.input_size, hidden_size)
    #     else:
    #         prev_hidden_size = self.hidden_layers[-1].out_features
    #         hidden_size = calculate_hidden_neurons(prev_hidden_size, self.output_size)
    #         new_layer = nn.Linear(prev_hidden_size, hidden_size)
    #     self.hidden_layers.append(new_layer)
    
    def forward(self, x):
        x = self.sigmoid(self.hidden_layer(x))
        x = self.sigmoid(self.output_layer(x))
        return x

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}')

In [ ]:
def evaluate_model(model, criterion, test_loader):
    test_loss = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(test_loader)}')

    return outputs

### Variáveis para o modelo, separação dos dados de teste e treino e etc

In [ ]:
train, test_data = train_test_split(features, test_size = 0.2, random_state = 2)
real_train_data, validation_data = train_test_split(train, test_size = 0.3, random_state = 2)

train_target, test_target = train_test_split(targets, test_size = 0.2, random_state = 2)
real_train_target, validation_target = train_test_split(train_target, test_size = 0.3, random_state = 2)

input_size = 603
output_size = 2
batch_size = 200
num_epochs = 150
learning_rate = 0.001

data_train = torch.tensor(real_train_data, dtype=torch.float32)
target_train = torch.tensor(real_train_target, dtype=torch.float32)

data_val = torch.tensor(validation_data, dtype=torch.float32)
target_val = torch.tensor(validation_target, dtype=torch.float32)

data_test = torch.tensor(test_data, dtype=torch.float32)
target_test = torch.tensor(test_target, dtype=torch.float32)

print("Input Shapes:")
print(data_train.shape, data_val.shape, data_test.shape)
print("Target Shapes:")
print(target_train.shape, target_val.shape, target_test.shape)

In [ ]:
# Data Loaders
train_dataset = torch.utils.data.TensorDataset(data_train, target_train)
val_dataset = torch.utils.data.TensorDataset(data_val, target_val)
test_dataset = torch.utils.data.TensorDataset(data_test, target_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = MLP(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and evaluate the model
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)
evaluate_model(model, criterion, test_loader)

torch.save(model.state_dict(), 'mlp_weights.pth')

In [ ]:
class GeneralizedMLP(nn.Module):
    def __init__(self, input_size, output_size, num_hidden_layers):
        super(GeneralizedMLP, self).__init__()
        hidden_neurons = calculate_hidden_neurons(input_size, output_size)
        
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(input_size, hidden_neurons))
        
        for _ in range(num_hidden_layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_neurons, hidden_neurons))
        
        self.output_layer = nn.Linear(hidden_neurons, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.sigmoid(layer(x))

        x = self.sigmoid(self.output_layer(x))
        return x

In [ ]:
saved_state_dict = torch.load('mlp_weights.pth')

# Initialize the new model
extended_model = GeneralizedMLP(input_size, output_size, 2)

# Copy the weights of the hidden_layer to hidden_layer1

extended_model.hidden_layers[0].load_state_dict({
    'weight': saved_state_dict['hidden_layer.weight'],
    'bias': saved_state_dict['hidden_layer.bias']
})

criterion = nn.MSELoss()
optimizer = optim.Adam(extended_model.parameters(), lr=learning_rate)

# Train and evaluate the model
train_model(extended_model, criterion, optimizer, train_loader, val_loader, num_epochs)
evaluate_model(extended_model, criterion, test_loader)

torch.save(extended_model.state_dict(), 'generalized_mlp_weights.pth')

In [ ]:
for i in range(1, 2):
    print(i)

In [ ]:
saved_state_dict = torch.load('generalized_mlp_weights.pth')

extended_model = GeneralizedMLP(input_size, output_size, 3)

extended_model.hidden_layers[0].load_state_dict({
    'weight': saved_state_dict['hidden_layers.0.weight'],
    'bias': saved_state_dict['hidden_layers.0.bias']
})

for i in range(1, 2):
    extended_model.hidden_layers[i].load_state_dict({
        'weight': saved_state_dict[f'hidden_layers.{i}.weight'],
        'bias': saved_state_dict[f'hidden_layers.{i}.bias']
})

criterion = nn.MSELoss()
optimizer = optim.Adam(extended_model.parameters(), lr=learning_rate)

# Train and evaluate the model
train_model(extended_model, criterion, optimizer, train_loader, val_loader, num_epochs)
evaluate_model(extended_model, criterion, test_loader)

torch.save(extended_model.state_dict(), 'generalized_mlp_weights.pth')


In [ ]:
saved_state_dict = torch.load('generalized_mlp_weights.pth')

extended_model = GeneralizedMLP(input_size, output_size, 4)

extended_model.hidden_layers[0].load_state_dict({
    'weight': saved_state_dict['hidden_layers.0.weight'],
    'bias': saved_state_dict['hidden_layers.0.bias']
})

for i in range(1, 3):
    extended_model.hidden_layers[i].load_state_dict({
        'weight': saved_state_dict[f'hidden_layers.{i}.weight'],
        'bias': saved_state_dict[f'hidden_layers.{i}.bias']
})
    
criterion = nn.MSELoss()
optimizer = optim.Adam(extended_model.parameters(), lr=learning_rate)

# Train and evaluate the model
train_model(extended_model, criterion, optimizer, train_loader, val_loader, num_epochs)
evaluate_model(extended_model, criterion, test_loader)

torch.save(extended_model.state_dict(), 'generalized_mlp_weights.pth')